<a href="https://colab.research.google.com/github/Noob-Bot-1/IML-2021/blob/main/IIT2019132_6c_WithRegularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
house_data = pd.read_csv("/content/drive/MyDrive/ML/Housing Price data set.csv")
house_data

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000.0,6360,2,1,1,yes,no,no,no,no,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,542,91500.0,4800,3,2,4,yes,yes,no,no,yes,0,no
542,543,94000.0,6000,3,2,4,yes,no,no,no,yes,0,no
543,544,103000.0,6000,3,2,4,yes,yes,no,no,yes,1,no
544,545,105000.0,6000,3,2,2,yes,yes,no,no,yes,1,no


In [3]:
house_data.columns

Index(['Unnamed: 0', 'price', 'lotsize', 'bedrooms', 'bathrms', 'stories',
       'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl',
       'prefarea'],
      dtype='object')

In [4]:
#Dividing each useful column from the data
Price = house_data['price']
FloorArea = house_data['lotsize']
NoOfBedrooms = house_data['bedrooms']
NoOfBathrooms = house_data['bathrms']

In [22]:
# Performing feature scanning on FloorArea
FloorArea_Mean = np.mean(FloorArea)
FloorArea_Max = max(FloorArea)
FloorArea_Min = min(FloorArea)
FloorArea_Scaled = []
for i in FloorArea:
	FloorArea_Scaled.append((i - FloorArea_Mean) / (FloorArea_Max - FloorArea_Min))

In [23]:
#Segmenting the features
FeaturesTrain = []
for i in range(500):
	FeaturesTrain.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
PriceTrain = Price[:500]
PriceTest = []
FeaturesTest = []
for i in range(500, len(Price)):
	FeaturesTest.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
	PriceTest.append(Price[i])
m = len(FeaturesTrain)

In [24]:
#Function to calculate Scaled Slope to find coefficients
def Slope(Coeff, FeaturesTrain, PriceTrain, ind):
	Error = 0
	for i in range(len(FeaturesTrain)):
		itr = 0
		for j in range(len(Coeff)):
			itr = itr + Coeff[j] * FeaturesTrain[i][j]
		Error += (itr - PriceTrain[i]) * FeaturesTrain[i][ind]
	return Error


In [36]:
# Using scaled batch gradient
print("Using scaled batch gradient")
LearningRate = 0.001
LambdaParameter = -200
Coeff = [0, 0, 0, 0]

for epochs in range(500):
	TempCoeff = Coeff.copy()
	for j in range(len(Coeff)):
		if (j == 0):
			TempCoeff[j] = TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))	
		else:
			TempCoeff[j] = (1 - LearningRate * LambdaParameter / m) * TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
	Coeff = TempCoeff.copy()
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))

Using scaled batch gradient
Final coefficients are:
[5159.936515597131, 969.4614637399126, 17169.065016048593, 9399.533115322818]
Mean absolute percentage error is : 19.67045546426377


In [32]:
#Function to calculate Stochastic Slope to find coefficients
def SlopeStoch(Coeff,FeaturesTrain,ActualVal,ind):
	itr = 0
	for j in range(len(Coeff)):
		itr = itr + Coeff[j]*FeaturesTrain[j]
	return (itr - ActualVal) * FeaturesTrain[ind]

In [49]:
# Using Scaled Stochastic gradient
print("Using Stochastic gradient")
LearningRate = 0.002
LambdaParameter = -200
Coeff = [0, 0, 0, 0]

y = LearningRate*LambdaParameter
y = 1-y
for iter in range(10):
	for i in range(len(PriceTrain)):
		for j in range(4):
			if j == 0:
				TempCoeff[j] = Coeff[j] - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
			else:
				TempCoeff[j] = (y*Coeff[j]) - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
		Coeff = TempCoeff.copy()
print("Final coefficients are:")
print(Coeff)


# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))

Using Stochastic gradient
Final coefficients are:
[nan, nan, nan, nan]
Mean absolute percentage error is : nan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [46]:
# Using Scaled Minibatch gradient for batch size = 30
print("Using Scaled Minibatch gradient for batch size = 30")

BatchSize = 30;
LearningRate = 0.002
LambdaParameter = -200
Coeff = [0, 0, 0, 0]
NoOfBatches = math.ceil(len(PriceTrain) / BatchSize)
equallyDiv = False
if (len(PriceTrain) % BatchSize == 0):
	equallyDiv = True;

for epoch in range(30):
	for batch in range(NoOfBatches):
		Summation = [0, 0, 0, 0]
		for j in range(len(Coeff)):
			for i in range(BatchSize):
				if (batch * BatchSize + i == len(FeaturesTrain)):
					break
				PredictedValue = 0.0
				for wj in range(len(Coeff)):
					PredictedValue += Coeff[wj] * FeaturesTrain[batch * BatchSize + i][wj]
				PredictedValue -= PriceTrain[batch * BatchSize + i]
				PredictedValue *= FeaturesTrain[batch * BatchSize + i][j]
				Summation[j] += PredictedValue;

		if (not equallyDiv and batch == NoOfBatches - 1):
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
		else:
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / BatchSize) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / BatchSize) * LearningRate
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Using Scaled Minibatch gradient for batch size = 30
Final coefficients are:
[4487.995623465271, 2114.253696359352, 16817.4711285676, 11641.033675783994]
Mean absolute percentage error is : 18.361536401321786

